<a href="https://colab.research.google.com/github/jeonggunlee/CUDATeaching/blob/master/02_cuda_lab/00_googleDrive_CUDAExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Drive를 활용한 CUDA 프로그래밍

직접 Google Drive에서 프로그램을 수행할 수 있습니다. 

이경우 mount가 필요하고 **여러개**의 화일로 이루어진 코드들을 수행할 수 있습니다.

또한 직접 구글 드라이브로 **git clone** 하여 소스를 살펴보고 실 데이터와 함께 수행 시킬 수 있습니다.

In [0]:
from google.colab import drive

Enter your authorization code:

지정된 URL을 클릭하면 인증 코드를 볼수 있으며, 이를 복사하여 입력합니다.

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd /content

/content


In [4]:
!ls

gdrive	sample_data


In [5]:
%cd gdrive

/content/gdrive


In [6]:
!ls

'My Drive'


In [7]:
%cd My Drive

/content/gdrive/My Drive


In [8]:
%cd GPUCoding

/content/gdrive/My Drive/GPUCoding


In [9]:
!ls 

cs344


In [0]:
!git clone https://github.com/jeonggunlee/cs344/

Cloning into 'cs344'...
remote: Enumerating objects: 726, done.
remote: Total 726 (delta 0), reused 0 (delta 0), pack-reused 726
Receiving objects: 100% (726/726), 66.62 MiB | 21.37 MiB/s, done.
Resolving deltas: 100% (402/402), done.
Checking out files: 100% (135/135), done.


In [0]:
!ls

cs344


In [0]:
%cd cs344

/content/gdrive/My Drive/GPUCoding/cs344


In [0]:
!ls

 CMakeLists.txt		'Lesson Slides'  'Student Contributions'
 Final			'Problem Sets'
'Lesson Code Snippets'	 README.md


In [0]:
%cd 'Lesson Code Snippets'

/content/gdrive/My Drive/GPUCoding/cs344/Lesson Code Snippets


In [0]:
!ls

'Lesson 2 Code Snippets'  'Lesson 5 Code Snippets'
'Lesson 3 Code Snippets'  'Lesson 7 Code Snippets'


In [0]:
%cd 'Lesson 2 Code Snippets'

/content/gdrive/My Drive/GPUCoding/cs344/Lesson Code Snippets/Lesson 2 Code Snippets


In [0]:
!ls

associative.cu	gputimer.h	   hello_threadIdx.cu
atomics.cu	hello_blockIdx.cu  memory.cu


In [0]:
!nvcc -o hello_threadIdx hello_threadIdx.cu

In [0]:
!./hello_threadIdx

Hello world! I'm thread 160
Hello world! I'm thread 161
Hello world! I'm thread 162
Hello world! I'm thread 163
Hello world! I'm thread 164
Hello world! I'm thread 165
Hello world! I'm thread 166
Hello world! I'm thread 167
Hello world! I'm thread 168
Hello world! I'm thread 169
Hello world! I'm thread 170
Hello world! I'm thread 171
Hello world! I'm thread 172
Hello world! I'm thread 173
Hello world! I'm thread 174
Hello world! I'm thread 175
Hello world! I'm thread 176
Hello world! I'm thread 177
Hello world! I'm thread 178
Hello world! I'm thread 179
Hello world! I'm thread 180
Hello world! I'm thread 181
Hello world! I'm thread 182
Hello world! I'm thread 183
Hello world! I'm thread 184
Hello world! I'm thread 185
Hello world! I'm thread 186
Hello world! I'm thread 187
Hello world! I'm thread 188
Hello world! I'm thread 189
Hello world! I'm thread 190
Hello world! I'm thread 191
Hello world! I'm thread 192
Hello world! I'm thread 193
Hello world! I'm thread 194
Hello world! I'm thr

In [0]:
!ls

associative.cu	gputimer.h	   hello_threadIdx     memory.cu
atomics.cu	hello_blockIdx.cu  hello_threadIdx.cu


In [0]:
!cat hello_threadIdx.cu

#include <stdio.h>

#define NUM_BLOCKS 1
#define BLOCK_WIDTH 256

__global__ void hello()
{
    printf("Hello world! I'm thread %d\n", threadIdx.x);
}


int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_WIDTH>>>();

    // force the printf()s to flush
    cudaDeviceSynchronize();

    printf("That's all!\n");

    return 0;
}


In [0]:
!ls

associative.cu	gputimer.h	   hello_threadIdx     memory.cu
atomics.cu	hello_blockIdx.cu  hello_threadIdx.cu


In [0]:
!nvcc -o memory memory.cu

memory.cu(11): warning: variable "f" was set but never used



In [0]:
!./memory

In [0]:
!nvcc -o associative associative.cu

In [0]:
!./associative

(1 + 1e+99) + -1e+99 == 0
1 + (1e+99 + -1e+99) == 1


In [0]:
!nvcc -o atomics atomics.cu

In [0]:
!./atomics

1000000 total threads in 1000 blocks writing into 100 array elements
{ 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 }
Time elapsed = 0.172832 ms
